<a href="https://colab.research.google.com/github/shubhamgupta1017/FishEye/blob/main/Phising_url_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from flask import Flask, request, jsonify
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse
import re

In [ ]:
df=pd.read_csv("/content/phishing_site_urls.csv")

In [ ]:
!pip install scikit-learn==1.4.0


In [ ]:
!pip install sklearn==1.4.0


ERROR: Could not find a version that satisfies the requirement sklearn==1.4.0 (from versions: 0.0, 0.0.post1, 0.0.post2, 0.0.post4, 0.0.post5, 0.0.post7, 0.0.post9, 0.0.post10, 0.0.post11, 0.0.post12)
ERROR: No matching distribution found for sklearn==1.4.0


In [ ]:
# data=np.array(df)

In [ ]:
# print(data)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549846 entries, 0 to 549845
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   URL     549846 non-null  object
 1   Label   549846 non-null  object
dtypes: object(2)
memory usage: 8.4+ MB


In [ ]:
# df['slash_count'] =''
# df['unicode'] = ''
# df['ip_address'] = ''
# df['at_symbol'] = ''
# df['dot_count'] = ''
# df['dash'] = ''
# df['double_slash'] = ''
# df['https_token'] = ''
# df['shortened_url'] = ''
# df['hostname_length'] = ''

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549846 entries, 0 to 549845
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   URL     549846 non-null  object
 1   Label   549846 non-null  object
dtypes: object(2)
memory usage: 8.4+ MB


In [ ]:
def count_slash(url):
    count = 0
    for char in url:
        if char == '/':
            count += 1
    return count

def c_unicode(url):
    parsed_url = urlparse(url)
    url_without_encoding = parsed_url.netloc.encode('idna').decode('utf-8')
    return url_without_encoding != parsed_url.netloc


def contains_ip_address(url):
    ip_pattern = r"\b(?:\d{1,3}\.){3}\d{1,3}\b"
    match = re.search(ip_pattern, url)
    return match is not None

def contains_at_symbol(url):
    return '@' in url


def count_dots_in_hostname(url):
    parsed_url = urlparse(url)
    hostname = parsed_url.hostname
    if hostname:
        return hostname.count('.')
    else:
        return 0

def contains_at_symbol(url):
    return '-' in url

def check_double_slash_in_path(url):
    parsed_url = urlparse(url)
    if parsed_url.path:
        if "//" in parsed_url.path:
            return 1
    return 0

def check_https_token_in_url(url):
    parsed_url = urlparse(url)
    if parsed_url.scheme == 'https':
        return 1
    return 0

def is_shortened_url(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc.lower() == 'tinyurl.com'

def check_hostname_length(url):
    if len(url) > 40:
            return 1
    return 0


def calc(url):
    try:
        # response = requests.get(url)
        # html_content = response.text
        features={}
        # soup = BeautifulSoup(html_content, 'html.parser')
        # features['link_count']=len(soup.find_all('a'))
        # features['i-frame']=int(bool(soup.find_all('iframe')))

        # script_tags = soup.find_all('script')
        # javascript_content = ''
        # for script in script_tags:
        #     script_content = script.text.lower()
        #     if 'event.button==2' in script_content or 'event.which==3' in script_content:
        #         if 'preventdefault' in script_content or 'return false' in script_content:
        #             features['right-dis'] = 1

        features['slash_count'] = int(count_slash(url))
        features['unicode'] = int(c_unicode(url))
        features['ip_address'] = int(contains_ip_address(url))
        features['at_symbol'] = int(contains_at_symbol(url))
        features['dot_count'] = int(count_dots_in_hostname(url))
        features['dash'] = int(contains_at_symbol(url))
        features['double_slash'] = int(check_double_slash_in_path(url))
        features['https_token'] = int(check_https_token_in_url(url))
        features['shortened_url'] = int(is_shortened_url(url))
        features['hostname_length'] = int(check_hostname_length(url))



        return features

    except Exception as e:
        print(f"Error occurred: {e}")
        return None, None


In [ ]:
features_list = [calc(url) for url in df['URL']]
print("..........................................................................................................................................")
# Convert the list of dictionaries into a DataFrame
features_df = pd.DataFrame(features_list)
print("..........................................................................................................................................")
# Concatenate the original DataFrame 'df' and the features DataFrame
df = pd.concat([df, features_df], axis=1)

..........................................................................................................................................
..........................................................................................................................................


In [ ]:
df.head()

,URL,Label,slash_count,unicode,ip_address,at_symbol,dot_count,dash,double_slash,https_token,shortened_url,hostname_length
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,10,0,0,1,0,1,0,0,0,1
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,4,0,0,1,0,1,0,0,0,1
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,11,0,0,1,0,1,0,0,0,1
3,mail.printakid.com/www.online.americanexpress....,bad,2,0,0,0,0,0,0,0,0,1
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,10,0,0,1,0,1,0,0,0,1


In [ ]:
dataset=df

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
encoded_values = label_encoder.fit_transform(dataset[["Label"]])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
dataset.drop(columns=['Label','URL'], inplace=True)
X=dataset
y=encoded_values

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549846 entries, 0 to 549845
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype
---  ------           --------------   -----
 0   slash_count      549846 non-null  int64
 1   unicode          549846 non-null  int64
 2   ip_address       549846 non-null  int64
 3   at_symbol        549846 non-null  int64
 4   dot_count        549846 non-null  int64
 5   dash             549846 non-null  int64
 6   double_slash     549846 non-null  int64
 7   https_token      549846 non-null  int64
 8   shortened_url    549846 non-null  int64
 9   hostname_length  549846 non-null  int64
dtypes: int64(10)
memory usage: 42.0 MB


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549846 entries, 0 to 549845
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype
---  ------           --------------   -----
 0   slash_count      549846 non-null  int64
 1   unicode          549846 non-null  int64
 2   ip_address       549846 non-null  int64
 3   at_symbol        549846 non-null  int64
 4   dot_count        549846 non-null  int64
 5   dash             549846 non-null  int64
 6   double_slash     549846 non-null  int64
 7   https_token      549846 non-null  int64
 8   shortened_url    549846 non-null  int64
 9   hostname_length  549846 non-null  int64
dtypes: int64(10)
memory usage: 42.0 MB


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create a Random Forest Classifier instance
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest model
rf_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
rf_classifier.predict(np.array([3,0,0,0,1,0,0,1,0,0]).reshape(1,-1))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0])

In [ ]:
def calci(url):
    try:
        # response = requests.get(url)
        # html_content = response.text
        array=[]
        # soup = BeautifulSoup(html_content, 'html.parser')
        # features['link_count']=len(soup.find_all('a'))
        # features['i-frame']=int(bool(soup.find_all('iframe')))

        # script_tags = soup.find_all('script')
        # javascript_content = ''
        # for script in script_tags:
        #     script_content = script.text.lower()
        #     if 'event.button==2' in script_content or 'event.which==3' in script_content:
        #         if 'preventdefault' in script_content or 'return false' in script_content:
        #             features['right-dis'] = 1

        array.append(int(count_slash(url)))
        array.append(int(c_unicode(url)))
        array.append(int(contains_ip_address(url)))
        array.append(int(contains_at_symbol(url)))
        array.append(int(count_dots_in_hostname(url)))
        array.append(int(contains_at_symbol(url)))
        array.append(int(check_double_slash_in_path(url)))
        array.append(int(check_https_token_in_url(url)))
        array.append(int(is_shortened_url(url)))
        array.append(int(check_hostname_length(url)))

        array=np.array(array)



        return array

    except Exception as e:
        print(f"Error occurred: {e}")
        return None, None


In [ ]:
from joblib import dump

# Assuming rf_classifier is your trained RandomForestClassifier model
dump(rf_classifier, 'random_forest_model.pkl')


['random_forest_model.pkl']

In [ ]:
from google.colab import files

# Download the .pkl file
files.download('random_forest_model.pkl')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>